In [11]:
import sqlite3

import pandas as pd


In [12]:
from interchange.logs.logger import Logger
from interchange.persistence.database import Database


log = Logger(__name__)


db = Database()

2025-12-03 09:03:01,983 :: PID 16528 :: TID 17804 :: database._create_connection :: Line 33 :: DEBUG :: Connected to SQLite database
2025-12-03 09:03:02,000 :: PID 16528 :: TID 17804 :: database._close_connection :: Line 44 :: DEBUG :: Closed connection to SQLite database


In [13]:
sql_statement = "DROP TABLE visa_rules_2;"
query = db._execute(sql_statement)

log.logger.info("Creating 'visa_rules_2' table")
db.create_table(
    table_name="visa_rules_2",
    fields_def={
        "jurisdiction": "TEXT",
        "region_country_code": "TEXT",
        "guide_date": "DATE",
        "valid_from": "DATE",
        "valid_until": "DATE",
        "fee_program": "TEXT",
        "intelica_id": "INT",
        "fpi": "TEXT",
        "fee_descriptor": "TEXT",
        "fee_description": "TEXT",
        "cod_hierarchy": "INT",
        "program_default": "TEXT",
        "fee_currency": "TEXT",
        "fee_variable": "NUMERIC",
        "fee_fixed": "NUMERIC",
        "fee_min": "NUMERIC",
        "fee_cap": "NUMERIC",
        "business_mode": "TEXT",
        "issuer_country": "TEXT",
        "issuer_region": "TEXT",
        "technology_indicator": "TEXT",
        "product_id": "TEXT",
        "fast_funds": "TEXT",
        "travel_indicator": "TEXT",
        "b2b_program_id": "TEXT",
        "account_funding_source": "TEXT",
        "nnss_indicator": "TEXT",
        "product_subtype": "TEXT",
        "transaction_code": "TEXT",
        "transaction_code_qualifier": "TEXT",
        "issuer_bin_8": "TEXT",
        "acquirer_bin": "TEXT",
        "acquirer_business_id": "TEXT",
        "transaction_amount_currency": "TEXT",
        "transaction_amount": "TEXT",
        "acquirer_country": "TEXT",
        "acquirer_region": "TEXT",
        "merchant_country_code": "TEXT",
        "merchant_country_region": "TEXT",
        "merchant_category_code": "TEXT",
        "requested_payment_service": "TEXT",
        "usage_code": "TEXT",
        "authorization_characteristics_indicator": "TEXT",
        "authorization_code": "TEXT",
        "pos_terminal_capability": "TEXT",
        "cardholder_id_method": "TEXT",
        "pos_entry_mode": "TEXT",
        "timeliness": "TEXT",
        "reimbursement_attribute": "TEXT",
        "special_condition_indicator": "TEXT",
        "fee_program_indicator": "TEXT",
        "moto_eci_indicator": "TEXT",
        "acceptance_terminal_indicator": "TEXT",
        "prepaid_card_indicator": "TEXT",
        "pos_environment_code": "TEXT",
        "business_format_code": "TEXT",
        "business_application_id": "TEXT",
        "type_purchase": "TEXT",
        "network_identification_code": "TEXT",
        "message_reason_code": "TEXT",
        "surcharge_amount": "TEXT",
        "authorized_amount": "TEXT",
        "authorization_response_code": "TEXT",
        "merchant_verification_value": "TEXT",
        "dynamic_currency_conversion_indicator": "TEXT",
        "cvv2_result_code": "TEXT",
        "national_tax_indicator": "TEXT",
        "merchant_vat": "TEXT",
        "summary_commodity": "TEXT",
        "processing_code_transaction_type": "TEXT",
        "point_of_service_condition_code": "TEXT",
        "cashback": "TEXT",
        "message_identifier": "TEXT",
        "validation_code": "TEXT",
        "v_i_p_full_financial_message_sets": "TEXT",
        "sender_data": "TEXT",
        "additional_sender_data": "TEXT",
        "settlement_service": "TEXT",
        "other_criteria_applies": "TEXT",
    },
)

2025-12-03 09:03:02,068 :: PID 16528 :: TID 17804 :: database._execute :: Line 57 :: DEBUG :: SQL statement executed successfully
2025-12-03 09:03:02,069 :: PID 16528 :: TID 17804 :: 1599005252.<module> :: Line 4 :: INFO :: Creating 'visa_rules_2' table
2025-12-03 09:03:02,070 :: PID 16528 :: TID 17804 :: database.create_table :: Line 88 :: DEBUG :: Attempting to execute CREATE TABLE SQL statement
2025-12-03 09:03:02,081 :: PID 16528 :: TID 17804 :: database._execute :: Line 57 :: DEBUG :: SQL statement executed successfully


In [14]:
route_visa = r"C:/GITHUB_REPOS/VISA Reglas Intercambio V32.xlsx"

In [15]:
def read_file_visa(route: str) -> pd.DataFrame:
    """Reads an excel file with visa format

    Args:
        route (str): path to excel file in local

    Returns:
        file (DataFrame): DataFrame with excel info.

    """
    file = pd.read_excel(route, sheet_name="Visa", dtype=str, skiprows=3)

    return file

In [16]:
def read_rules_visa(dataframe: pd.DataFrame):
    """VISA Rules Reader

    Args:
        dataframe (DataFrame): DataFrame with data.
        name_log (str): name of log file.

    Returns:
        data (DataFrame): dataframe with data processed.

    """
    dataframe = dataframe.rename(
        columns={"DYNAMIC_CURRENCY_INDICATOR": "DYNAMIC_CURRENCY_CONVERSION_INDICATOR"}
    )
    data = dataframe.copy()

    if data["JURISDICTION"].isnull().any():
        raise

    if data["GUIDE_DATE"].isnull().any():
        raise

    if data["VALID_FROM"].isnull().any():
        raise

    if data["FEE_PROGRAM"].isnull().any():
        raise

    if data["INTELICA_ID"].isnull().any():
        raise

    if data["FPI"].isnull().any():
        raise

    if data["FEE_DESCRIPTOR"].isnull().any():
        raise

    if data["FEE_DESCRIPTION"].isnull().any():
        raise

    if data["COD_HIERARCHY"].isnull().any():
        raise

    data["FEE_FIXED"] = data["FEE_FIXED"].apply(lambda x: float(x))

    data["FEE_MIN"] = data["FEE_MIN"].apply(lambda x: float(x))

    data["FEE_CAP"] = data["FEE_CAP"].apply(lambda x: float(x))

    data["FEE_VARIABLE"] = data["FEE_VARIABLE"].apply(lambda x: float(x))

    data = data.copy()

    data.columns = data.columns.str.lower()

    return data

In [17]:
data = read_file_visa(route_visa)

In [ ]:
data_ingest = read_rules_visa(data)

In [ ]:
cols_fecha = ["guide_date", "valid_from", "valid_until"]

for col in cols_fecha:
    data_ingest[col] = (
        pd.to_datetime(data_ingest[col], errors="coerce")  # convierte a datetime
        .dt.strftime("%Y-%m-%d")  # formatea
        .replace("NaT", "")  # vacío si no es fecha
    )

data_ingest = data_ingest.fillna("")


In [ ]:
conn = sqlite3.connect("persistence/backup/interchange.db")

# Insertar el DataFrame creando una tabla nueva
data_ingest.to_sql(
    name="visa_rules_2",  # nombre de la nueva tabla
    con=conn,
    if_exists="append",  # Crea la tabla de cero (drop + create)
    index=False,
)

conn.close()